# Forecasting Churn of Customers 

**Initial Data Download**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import sklearn
from datetime import datetime
import joblib

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [2]:
# Paths
contract_path = "contract.csv"
personal_path = "personal.csv"
internet_path = "internet.csv"
phone_path = "phone.csv"

In [3]:
# Load
contract = pd.read_csv(contract_path)
personal = pd.read_csv(personal_path)
internet = pd.read_csv(internet_path)
phone = pd.read_csv(phone_path)

**Preprocessing**

In [4]:
# Configuration
# Config
SNAPSHOT = pd.to_datetime("2020-02-01")
SEED = 42
RAW_DIR = "data/raw"
PROC_DIR = "data/processed"
MODELS_DIR = "models"
os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(PROC_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)

# Input file names 
contract_path = os.path.join(RAW_DIR, "contract.csv")
personal_path = os.path.join(RAW_DIR, "personal.csv")
internet_path = os.path.join(RAW_DIR, "internet.csv")
phone_path = os.path.join(RAW_DIR, "phone.csv")

# safe read with fallback encoding
def safe_read_csv(path, **kwargs):
    if not os.path.exists(path):
        raise FileNotFoundError(f"File not found: {path} -- please put the CSV in {RAW_DIR}")
    try:
        return pd.read_csv(path, low_memory=False, **kwargs)
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding="latin-1", low_memory=False, **kwargs)

# months between series date and snapshot (days/30.44)
def months_between(series_dates, ref_date=SNAPSHOT):
    days = (ref_date - series_dates).dt.days
    months = days / 30.44
    return months.clip(lower=0)

In [5]:
# Normalize column names (strip)
for df in (contract, personal, internet, phone):
    df.columns = df.columns.str.strip()

# Quick shape and head
print("Shapes:")
print("contract:", contract.shape)
print("personal:", personal.shape)
print("internet:", internet.shape)
print("phone:", phone.shape)
print("\nSample (contract):")
display(contract.head(3))

# Check customerID counts and duplicates
for name, df in [("contract", contract), ("personal", personal), ("internet", internet), ("phone", phone)]:
    if "customerID" in df.columns:
        uniq = df["customerID"].nunique()
        dups = df.shape[0] - uniq
        print(f"{name}: rows={df.shape[0]}, unique customerID={uniq}, duplicates={dups}")
    else:
        print(f"WARNING: {name} missing 'customerID' column")

# Check EndDate raw values (contract)
if "EndDate" in contract.columns:
    print("\ncontract['EndDate'] value counts (top 20):")
    display(contract["EndDate"].astype(str).value_counts(dropna=False).head(20))
else:
    print("WARNING: contract has no 'EndDate' column; check the CSV.")

Shapes:
contract: (7043, 8)
personal: (7043, 5)
internet: (5517, 8)
phone: (6361, 2)

Sample (contract):


,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,7590-VHVEG,2020-01-01,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5
2,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15


contract: rows=7043, unique customerID=7043, duplicates=0
personal: rows=7043, unique customerID=7043, duplicates=0
internet: rows=5517, unique customerID=5517, duplicates=0
phone: rows=6361, unique customerID=6361, duplicates=0

contract['EndDate'] value counts (top 20):


No                     5174
2019-11-01 00:00:00     485
2019-12-01 00:00:00     466
2020-01-01 00:00:00     460
2019-10-01 00:00:00     458
Name: EndDate, dtype: int64

In [6]:
# Parse BeginDate; keep EndDate raw then parse non-'No' values
contract["BeginDate"] = pd.to_datetime(contract["BeginDate"], errors="coerce")
contract["EndDate_raw"] = contract["EndDate"].astype(str).fillna("No")  # keep as string
# replace 'No' with pd.NaT for parsing (but keep raw)
contract["EndDate_parsed"] = contract["EndDate_raw"].replace({"No": pd.NaT})
contract["EndDate_parsed"] = pd.to_datetime(contract["EndDate_parsed"], errors="coerce")

# Coerce numeric columns
for col in ["MonthlyCharges", "TotalCharges"]:
    if col in contract.columns:
        contract[col] = pd.to_numeric(contract[col], errors="coerce")

# Compute tenure (snapshot-based) and optional tenure until end for EDA (do NOT use tenure_until_end in modeling)
contract["tenure_months_snapshot"] = months_between(contract["BeginDate"], SNAPSHOT).round(2)
contract["tenure_months_until_end"] = months_between(contract["BeginDate"], contract["EndDate_parsed"]).round(2)

# Report naive missing counts
print("Missing counts (contract):")
print(contract[["BeginDate", "EndDate_raw", "EndDate_parsed", "MonthlyCharges", "TotalCharges", "tenure_months_snapshot"]].isna().sum())

Missing counts (contract):
BeginDate                    0
EndDate_raw                  0
EndDate_parsed            5174
MonthlyCharges               0
TotalCharges                11
tenure_months_snapshot       0
dtype: int64


In [7]:
# Fill missing TotalCharges deterministically where possible
initial_missing_total = contract["TotalCharges"].isna().sum()
print("Initial missing TotalCharges:", initial_missing_total)

# Approximate total = MonthlyCharges * tenure_months_snapshot (where MonthlyCharges present)
mask_fill = contract["TotalCharges"].isna() & contract["MonthlyCharges"].notna()
approx_total = (contract.loc[mask_fill, "MonthlyCharges"] * contract.loc[mask_fill, "tenure_months_snapshot"]).round(2)
contract.loc[mask_fill, "TotalCharges"] = approx_total

after_first_fill_missing = contract["TotalCharges"].isna().sum()
print("Missing after Monthly*tenure fill:", after_first_fill_missing)

# Fallback: fill remaining missing with MonthlyCharges (reasonable for new customers)
mask_fallback = contract["TotalCharges"].isna() & contract["MonthlyCharges"].notna()
contract.loc[mask_fallback, "TotalCharges"] = contract.loc[mask_fallback, "MonthlyCharges"]

after_fallback_missing = contract["TotalCharges"].isna().sum()
print("Missing after fallback fill:", after_fallback_missing)

# Final fallback: if still missing, fill with median TotalCharges (rare)
if contract["TotalCharges"].isna().any():
    med = contract["TotalCharges"].median()
    contract["TotalCharges"].fillna(med, inplace=True)
    print("Filled remaining with median:", med)

print("TotalCharges dtype and sample values:")
print(contract["TotalCharges"].dtype)
display(contract[["MonthlyCharges", "TotalCharges", "tenure_months_snapshot"]].head(5))

Initial missing TotalCharges: 11
Missing after Monthly*tenure fill: 0
Missing after fallback fill: 0
TotalCharges dtype and sample values:
float64


,MonthlyCharges,TotalCharges,tenure_months_snapshot
0,29.85,29.85,1.02
1,56.95,1889.50,34.03
2,53.85,108.15,4.04
3,42.30,1840.75,45.04
4,70.70,151.65,5.03


In [8]:
# Merge tables (left join on contract)
merged = contract.merge(personal, on="customerID", how="left", suffixes=("", "_personal"))
merged = merged.merge(internet, on="customerID", how="left", suffixes=("", "_internet"))
merged = merged.merge(phone, on="customerID", how="left", suffixes=("", "_phone"))

print("Merged shape:", merged.shape)
# Save a raw merged snapshot for traceability
merged_raw_path = os.path.join(PROC_DIR, "merged_raw_snapshot.csv")
merged.to_csv(merged_raw_path, index=False)
print("Saved merged raw snapshot to:", merged_raw_path)

# Show number of rows with missing personal/internet/phone info
cols_to_check = ["gender", "InternetService", "MultipleLines"]
for c in cols_to_check:
    if c in merged.columns:
        miss = merged[c].isna().sum()
        print(f"Column {c}: missing {miss} ({miss/len(merged):.2%})")

Merged shape: (7043, 24)
Saved merged raw snapshot to: data/processed/merged_raw_snapshot.csv
Column gender: missing 0 (0.00%)
Column InternetService: missing 1526 (21.67%)
Column MultipleLines: missing 682 (9.68%)


In [9]:
# Normalize common values and create binary flags
def yn_map(val):
    if pd.isna(val):
        return np.nan
    s = str(val).strip().lower()
    if s in ("yes", "y", "true", "1"):
        return 1
    if s in ("no", "n", "false", "0", "no internet service", "no phone service"):
        return 0
    return np.nan

# List of known binary service columns (found across data)
service_cols = [
    "PaperlessBilling",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
    "MultipleLines",
    "Partner",
    "Dependents"
]

for c in service_cols:
    if c in merged.columns:
        merged[c + "_bin"] = merged[c].apply(yn_map).astype("Int64")

# Convert SeniorCitizen to numeric if present (some datasets already 0/1)
if "SeniorCitizen" in merged.columns:
    merged["SeniorCitizen"] = pd.to_numeric(merged["SeniorCitizen"], errors="coerce").fillna(0).astype(int)

# Standardize small categorical columns by stripping whitespace
for c in ["Type", "PaymentMethod", "InternetService", "gender"]:
    if c in merged.columns:
        merged[c] = merged[c].astype(str).str.strip().replace({"nan": np.nan})

In [10]:
# Derived: number of internet services enabled (sum of binary internet flags)
internet_bin_cols = [c for c in merged.columns if c.startswith(("OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies")) and c.endswith("_bin")]
if internet_bin_cols:
    merged["num_internet_services"] = merged[internet_bin_cols].fillna(0).sum(axis=1).astype(int)
else:
    merged["num_internet_services"] = 0

# has_internet: true if InternetService is not null and not 'No'
merged["has_internet"] = (~merged["InternetService"].isna()) & (merged["InternetService"].str.lower() != "no")
merged["has_internet"] = merged["has_internet"].astype(int)

# has_phone: if MultipleLines is present and not 'No' treat as phone present
if "MultipleLines" in merged.columns:
    merged["has_phone"] = merged["MultipleLines"].apply(lambda v: 0 if pd.isna(v) else (0 if str(v).strip().lower() in ("no", "no phone service") else 1)).astype("Int64").fillna(0).astype(int)
else:
    merged["has_phone"] = 0

# num_services: internet services + phone presence
merged["num_services"] = merged["num_internet_services"] + merged["has_phone"]

# avg_monthly_from_total: TotalCharges / tenure (use MonthlyCharges fallback)
merged["avg_monthly_from_total"] = merged["TotalCharges"] / merged["tenure_months_snapshot"].replace({0: np.nan})
merged["avg_monthly_from_total"] = merged["avg_monthly_from_total"].replace([np.inf, -np.inf], np.nan).fillna(merged["MonthlyCharges"])

# payment_auto flag (PaymentMethod contains 'automatic')
merged["payment_auto"] = merged["PaymentMethod"].fillna("").str.lower().str.contains("automatic").astype(int)

# Show derived columns sample
display(merged[["tenure_months_snapshot", "MonthlyCharges", "TotalCharges", "num_internet_services", "has_internet", "has_phone", "num_services", "avg_monthly_from_total", "payment_auto"]].head(5))

,tenure_months_snapshot,MonthlyCharges,TotalCharges,num_internet_services,has_internet,has_phone,num_services,avg_monthly_from_total,payment_auto
0,1.02,29.85,29.85,1,1,0,1,29.264706,0
1,34.03,56.95,1889.50,2,1,0,2,55.524537,0
2,4.04,53.85,108.15,2,1,0,2,26.769802,0
3,45.04,42.30,1840.75,3,1,0,3,40.869227,1
4,5.03,70.70,151.65,0,1,0,0,30.149105,0


In [11]:
# Label columns
# churn_left = 1 if EndDate_parsed is present (customer left before snapshot)
merged["churn_left"] = merged["EndDate_parsed"].notna().astype(int)
# active_at_snapshot = 1 if EndDate_raw == 'No' (customer active at snapshot)
merged["active_at_snapshot"] = merged["EndDate_raw"].astype(str).str.lower().eq("no").astype(int)

# Default target: follow Clarification Summary shown to -> EndDate == 'No'
chosen_target = "active_at_snapshot"   

print("Target columns distribution:")
print("churn_left:")
display(merged["churn_left"].value_counts(normalize=True))
print("active_at_snapshot:")
display(merged["active_at_snapshot"].value_counts(normalize=True))

print(f"\nDefault chosen_target = '{chosen_target}'. To switch, change the chosen_target variable to 'churn_left' when training.")
# Add final target column for convenience
merged["target"] = merged[chosen_target].astype(int)
print("Final target distribution (merged['target']):")
display(merged["target"].value_counts(normalize=True))

Target columns distribution:
churn_left:


0    0.73463
1    0.26537
Name: churn_left, dtype: float64

active_at_snapshot:


1    0.73463
0    0.26537
Name: active_at_snapshot, dtype: float64


Default chosen_target = 'active_at_snapshot'. To switch, change the chosen_target variable to 'churn_left' when training.
Final target distribution (merged['target']):


1    0.73463
0    0.26537
Name: target, dtype: float64

In [12]:
# Columns to keep in cleaned dataset 
keep_cols = [
    "customerID",
    "BeginDate",
    "EndDate_raw",
    "EndDate_parsed",
    "tenure_months_snapshot",
    "MonthlyCharges",
    "TotalCharges",
    "num_internet_services",
    "has_internet",
    "has_phone",
    "num_services",
    "avg_monthly_from_total",
    "payment_auto",
    "gender",
    "SeniorCitizen",
    "Partner",
    "Dependents",
    "Type",
    "PaperlessBilling",
    "PaymentMethod",
    "InternetService",
    "MultipleLines",
    "churn_left",
    "active_at_snapshot",
    "target"
]
# Keep only existing columns
keep_cols = [c for c in keep_cols if c in merged.columns]
cleaned = merged[keep_cols].copy()

# Save cleaned CSV
cleaned_path = os.path.join(PROC_DIR, "merged_clean.csv")
cleaned.to_csv(cleaned_path, index=False)
print("Saved cleaned dataset to:", cleaned_path)

# Build an unfitted ColumnTransformer template
numeric_features = [c for c in ["MonthlyCharges", "TotalCharges", "tenure_months_snapshot", "num_internet_services", "num_services", "avg_monthly_from_total", "SeniorCitizen"] if c in cleaned.columns]
categorical_features = [c for c in ["Type", "PaymentMethod", "InternetService", "gender"] if c in cleaned.columns]

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Create OneHotEncoder in a cross-version-compatible way
from sklearn.preprocessing import OneHotEncoder
onehot_kwargs = {"handle_unknown": "ignore"}
# newer sklearn: OneHotEncoder accepts sparse_output, older one uses sparse
try:
    # prefer sparse_output if available
    ohe = OneHotEncoder(**onehot_kwargs, sparse_output=False)
except TypeError:
    
    ohe = OneHotEncoder(**onehot_kwargs, sparse=False)

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
    ("onehot", ohe)
])

preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
], remainder="drop", sparse_threshold=0)

# Save the unfitted template (so it can be fitted as part of model pipeline)
preproc_template_path = os.path.join(MODELS_DIR, "preprocessor_template.joblib")
joblib.dump(preprocessor, preproc_template_path)
print("Saved preprocessor template to:", preproc_template_path)

# Also save cleaned df as joblib for quick loading later
cleaned_joblib_path = os.path.join(PROC_DIR, "merged_clean_df.joblib")
joblib.dump(cleaned, cleaned_joblib_path)
print("Saved cleaned dataframe (joblib) to:", cleaned_joblib_path)

Saved cleaned dataset to: data/processed/merged_clean.csv
Saved preprocessor template to: models/preprocessor_template.joblib
Saved cleaned dataframe (joblib) to: data/processed/merged_clean_df.joblib
